# Data Indexing and Selection

Hemos visto en detalle los métodos y herramientas para acceder, establecer y modificar valores en matrices NumPy.
Estos incluyen indexación (por ejemplo, ``arr[2, 1]``), corte (por ejemplo, ``arr[:, 1:5]``), enmascaramiento (por ejemplo, ``arr[arr > 0]``), indexación "anidada" (por ejemplo, ``arr[0, [1, 5]]``), y combinaciones de los mismos (por ejemplo, ``arr[:, [1, 5]]``).


**Aquí veremos formas similares de acceder y modificar valores en objetos Pandas ``Series`` y ``DataFrame``.
Si has usado los patrones de NumPy, los patrones correspondientes en Pandas te resultarán muy familiares, aunque hay algunas peculiaridades que debes tener en cuenta.

Empezaremos con el caso simple del objeto unidimensional ``Series``, y luego pasaremos al más complicado objeto bidimensional ``DataFrame``.

## Selección de datos en serie

Como vimos en la sección anterior, un objeto **``Series`` actúa en muchos aspectos como un array unidimensional de NumPy, y en muchos aspectos como un diccionario estándar de Python.**
Si mantenemos en mente estas dos analogías superpuestas, nos ayudará a entender los patrones de indexación y selección de datos en estos arrays.

### Series como diccionario

Al igual que un diccionario, el objeto ``Series`` proporciona una correspondencia entre una colección de claves y una colección de valores:

In [7]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [3]:
data['b']

0.5

También podemos utilizar expresiones y métodos de Python tipo diccionario para examinar las claves/índices y los valores:

In [4]:
'b' in data

True

In [5]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [5]:
data.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [6]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

Los objetos ``Series`` pueden incluso modificarse con una sintaxis similar a la de un diccionario.
Del mismo modo que se puede ampliar un diccionario asignando una nueva clave, se puede ampliar una ``Serie`` asignando un nuevo valor de índice:

In [7]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

Esta fácil mutabilidad de los objetos es una característica conveniente: bajo el capó, Pandas está tomando decisiones sobre la disposición de la memoria y la copia de datos que pueda ser necesario realizar; el usuario generalmente no necesita preocuparse por estos temas.

### Series como array unidimensional

Una ``Serie`` se basa en esta interfaz tipo diccionario y proporciona una selección de elementos tipo array a través de los mismos mecanismos básicos que los arrays de NumPy, es decir, *slices*, *enmascaramiento* e *índice "anidados"*.
Algunos ejemplos son los siguientes:

In [8]:
# corte por índice explícito
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [9]:
# corte por índice entero implícito
data[0:2]

a    0.25
b    0.50
dtype: float64

In [ ]:
data

In [13]:
# enmascaramiento
# Esta va a ser muy importante con DataFrames
# Nótese que el operador lógico es & y no and.
data[(data > 0.3) & (data < 0.8)]

data[(data>0.5)&(data<=1)]

data[(data==0.5)|(data==1)]

b    0.5
d    1.0
dtype: float64

In [11]:
# indexación "anidada"
data[['a', 'e']]



a    0.25
e    1.25
dtype: float64

Entre ellos, el troceado puede ser la fuente de mayor confusión.

**Tenga en cuenta que cuando se corta con un índice explícito (es decir, ``data['a':'c']``), el índice final está *incluido* en la rebanada, mientras que cuando se corta con un índice implícito (es decir, ``data[0:2]``), el índice final está *excluido* de la rebanada.**

In [12]:
# corte por índice explícito
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [13]:
# corte por índice entero implícito
data[0:2]

a    0.25
b    0.50
dtype: float64

### Indexadores: loc e iloc

Estas convenciones de corte e indexación pueden ser fuente de confusión.
Por ejemplo, si tu ``Serie`` tiene un índice entero explícito, una operación de indexación como **`datos[1] ``usará los índices explícitos, mientras que una operación de corte como ``datos[1:3]`` usará el índice implícito al estilo Python.**

In [6]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data



TypeError: '>' not supported between instances of 'str' and 'float'

In [15]:
# explicit index when indexing
data[1]

'a'

In [17]:
# implicit index when slicing
data[1:3]

3    b
5    c
dtype: object

In [19]:
# Si te pasas, se queda en el final
data[1:321]

3    b
5    c
dtype: object

Debido a esta confusión potencial en el caso de índices enteros, Pandas proporciona algunos atributos *indexer* especiales que exponen explícitamente ciertos esquemas de indexación.
No se trata de métodos funcionales, sino de atributos que exponen una interfaz de corte particular para los datos de la ``Serie``.

En primer lugar, el atributo **`loc`` permite la indexación y el corte que siempre hace referencia al índice explícito:**

In [21]:
data

1    a
3    b
5    c
dtype: object

In [22]:
data.loc[1]

'a'

In [25]:
data.loc[1:3]

1    a
3    b
dtype: object

El atributo ``iloc`` permite indexar y slicing que siempre hace referencia al índice implícito estilo Python:

In [26]:
data

1    a
3    b
5    c
dtype: object

In [27]:
data.iloc[1]

'b'

In [29]:
data.iloc[1:2]

3    b
dtype: object

Aunque en estos ejemplos de juguete .loc y .iloc paracen hasta cierto punto intercambiables, siempre que conozcas la equivalencia entre el índice explícito e implícito.
Para la mayoría de los proyectos de data science, .loc tiene la ventaja de preservar el resultado de la selección incluso si el orden/tamaño del objeto pandas cambia! (con ciertas restricciones, como por ejemplo que dichos indices no vuelvan a aparecer en los nuevos datos)

Teniendo en cuenta que es muy frecuente generar un orden aleatorio de los datos (salvo en Time Series), si nos importa el dato concreto al que nos estamos refiriendo y no queremos que sea aleatorio, loc es mucho más "seguro" que iloc 

In [47]:
data = pd.Series(['a', 'b', 'c', 'd'], index=[1, 3, 5, 7])

In [49]:
print(data.loc[1])
print(data.iloc[1])

a
b


In [50]:
print(data.sample(3, random_state=42).loc[1])
print(data.sample(3, random_state=42).iloc[1])

a
d


Lo mismo sucede si hacemos subsets: iloc se "reestructura" y ahora la posición 1, la ocupa 'c'

In [66]:
data_subset = data[[1,5]].copy()
print(data_subset.loc[1])
print(data_subset.iloc[1])

a
c


Un principio rector del código Python es que "lo explícito es mejor que lo implícito".
La naturaleza explícita de ``loc`` y ``iloc`` los hace muy útiles para mantener un código limpio y legible; especialmente en el caso de índices de enteros, **se recomienda usar los para hacer el código más fácil de leer y entender, así como para para prevenir bugs sutiles debidos a la convención mixta de indexación/slicing.**

## Selección de datos en DataFrame

Recordemos que un ``DataFrame`` actúa en muchos aspectos como una matriz bidimensional o estructurada, y en otros como un diccionario de estructuras ``Series`` que comparten el mismo índice.
Estas analogías pueden ser útiles para tener en cuenta a medida que exploramos la selección de datos dentro de esta estructura.

### DataFrame como diccionario

La primera analogía que vamos a considerar es el ``DataFrame`` como diccionario de objetos ``Series`` relacionados.
Volvamos a nuestro ejemplo de las áreas y poblaciones de los estados:

In [67]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})

data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


Se puede acceder a las ``Series`` individuales que componen las columnas del ``DataFrame`` mediante la indexación de tipo diccionario del nombre de la columna:

In [68]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

De forma equivalente, podemos utilizar el acceso de tipo atributo con nombres de columna que sean cadenas:

In [73]:
data.area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Este acceso de columna de tipo atributo accede en realidad exactamente al mismo objeto que el acceso de tipo diccionario:

In [71]:
data.area is data['area']

True

Aunque se trata de una abreviatura útil, hay que tener en cuenta que no funciona en todos los casos.
Por ejemplo, **si los nombres de las columnas no son cadenas, o si los nombres de las columnas entran en conflicto con métodos del ``DataFrame``, este acceso tipo atributo no es posible.**
Por ejemplo, el ``DataFrame`` tiene un método ``pop()``, por lo que ``data.pop`` apuntará a éste en lugar de a la columna ``"pop"``:

In [72]:
data.pop is data['pop']

False

In [ ]:
# La documentacion no engaña
data.pop

En particular, debe evitar la tentación de intentar asignar columnas mediante atributos (es decir, utilice ``datos['pop'] = z`` en lugar de ``datos.pop = z``).

Al igual que con los objetos ``Series`` comentados anteriormente, esta sintaxis de tipo diccionario también se puede utilizar para modificar el objeto, en este caso añadiendo una nueva columna:

In [75]:
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [74]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Esto muestra una vista previa de la sintaxis directa de la aritmética elemento a elemento entre objetos ``Series``; profundizaremos más en esto en [Operando con datos en Pandas](3_Operaciones-en-Pandas.ipynb).

### DataFrame como matriz bidimensional

Como se mencionó anteriormente, también podemos ver el ``DataFrame`` como una matriz bidimensional mejorada.
Podemos examinar la matriz de datos subyacente utilizando el atributo ``values``:

In [76]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

Con esta imagen en mente, se pueden hacer muchas observaciones familiares sobre el propio ``DataFrame``.
**Por ejemplo, podemos transponer el ``DataFrame`` completo para intercambiar filas y columnas:**.

In [77]:
data.T

,California,Texas,New York,Florida,Illinois
area,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
pop,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01


Cuando se trata de la indexación de objetos ``DataFrame``, sin embargo, está claro que el estilo de indexación de diccionario de las columnas impide nuestra **capacidad de simplemente tratarlo como una matriz NumPy.**
En particular, al pasar un único índice a un array se accede a una fila:

In [98]:
data.values[0]

array([4.23967000e+05, 3.83325210e+07, 9.04139261e+01])

mientras que en un``DataFrame``, se accede a la columna:

In [81]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Por lo tanto, para la indexación estilo array, necesitamos otra convención.
Aquí Pandas utiliza de nuevo los indexadores ``loc``, ``iloc``, y ``ix`` mencionados anteriormente.
Usando el indexador ``iloc``, podemos indexar el array subyacente como si fuera un simple array NumPy (usando el índice implícito estilo Python), pero el índice ``DataFrame`` y las etiquetas de las columnas se mantienen en el resultado:

In [15]:
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [14]:
data.iloc[:4, 1:3]

IndexingError: Too many indexers

In [96]:
data.values[:4, 1:3]

array([[3.83325210e+07, 9.04139261e+01],
       [2.64481930e+07, 3.80187404e+01],
       [1.96511270e+07, 1.39076746e+02],
       [1.95528600e+07, 1.14806121e+02]])

In [97]:
data[:4, 1:3]

InvalidIndexError: (slice(None, 4, None), slice(1, 3, None))

Del mismo modo, utilizando el indexador ``loc`` podemos indexar los datos subyacentes en un estilo similar a un array pero utilizando el índice explícito y los nombres de las columnas:

In [99]:
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [100]:
data.loc['Texas':'New York', 'pop':'density']

,pop,density
Texas,26448193,38.018740
New York,19651127,139.076746


**IX ESTÁ OBSOLETO**


![imagen](./img/in_ictu_oculi.jpg)


Recordatorio de lo efímera que es la vida (del código). 

El indexador ``ix`` permitía un híbrido de estos dos enfoques:
https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.ix.html


Dentro de estos indexadores se puede utilizar cualquiera de los patrones de acceso a datos conocidos del estilo NumPy.
Por ejemplo, en el indexador ``loc`` podemos combinar enmascaramiento e indexación "anidada" como en lo siguiente:

In [4]:
import pandas as pd
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})

data = pd.DataFrame({'area':area, 'pop':pop})
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [5]:
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
New York,19651127,139.076746
Florida,19552860,114.806121


Cualquiera de estas convenciones de indexación también se puede utilizar para establecer o modificar valores; esto se hace de la manera estándar a la que puede estar acostumbrado de trabajar con NumPy:

In [6]:
data.iloc[0, 2] = 90
data

,area,pop,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Para mejorar tu fluidez en la manipulación de datos en Pandas, puedes pasar algún tiempo con un ``DataFrame`` y explores los tipos de indexación, troceado, enmascaramiento e indexación "anidada" que permiten estos diversos enfoques de indexación.

Si además, dicho ``DataFrame`` procede de datos con los que tienes familiaridad, podrás empezar a descubrir el potencial de este nuevo objeto de Python

### Convenciones de indexación adicionales

Hay un par de convenciones de indexación adicionales que pueden parecer contradictorias con la discusión anterior, pero que sin embargo pueden ser muy útiles en la práctica.
**En primer lugar, mientras que *indexar* se refiere a columnas, *slicing* se refiere a filas:**.

In [7]:
data['Florida':'Illinois']

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [8]:
data.loc[:, 'area':'density']

,area,pop,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Estas slices también pueden referirse a filas por número en lugar de por índice:

In [9]:
data[1:3]

,area,pop,density
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746


Del mismo modo, las operaciones de enmascaramiento directo también se interpretan por filas en lugar de por columnas:

In [10]:
data[data.density > 100]

,area,pop,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
